# Predicting Loan Default 3 - Models

In this notebook we perform model fitting, evaluation and selection. 

## Packages

In [1]:
## data handling 
import numpy as np 
import pandas as pd
import polars as pl 
import polars.selectors as cs

## tuning
import optuna 

## visualization
import matplotlib.pyplot as plt
import seaborn as sns
import plotly
import plotly.express as px

## sklearn models
from sklearn.model_selection import train_test_split, StratifiedKFold
from sklearn.linear_model import LogisticRegression
from sklearn.tree import DecisionTreeClassifier
from sklearn.neighbors import KNeighborsClassifier
from sklearn.naive_bayes import GaussianNB
from sklearn.neural_network import MLPClassifier
from sklearn.ensemble import AdaBoostClassifier, GradientBoostingClassifier, RandomForestClassifier
from sklearn.metrics import classification_report, confusion_matrix, accuracy_score
from sklearn.model_selection import cross_val_score
from sklearn.preprocessing import StandardScaler

## pickle 
from pickle import dump, load

## get file path of the data
from private import ENCODED_FILE_PATH, FINAL_FILE_PATH

## Data

In [2]:
## load datasets
X_train = pd.read_csv(FINAL_FILE_PATH + "X_train.csv", index_col=0)
X_test = pd.read_csv(FINAL_FILE_PATH + "X_test.csv", index_col=0)
y_train = pd.read_csv(FINAL_FILE_PATH + "y_train.csv", index_col=0)
y_test = pd.read_csv(FINAL_FILE_PATH + "y_test.csv", index_col=0)

## Models

### Model Evaluation 

Due to the imbalanced nature of our `default` feature, we must carefully consider our metric. As false negatives (borrowers who default but who were predicted to not default) are much more costly to investors than false positives (borrowers who don't default but who were predicted to default), we will make use of recall as a metric, as well as the area under the ROC curve. 

### Model Selection 

We will use 3-fold cross validation on a large number of models with default parameters. This will help gauge which type of model may be performing best. We will then select the best performing model for hyperparameter tuning. 

In [3]:
models = []

## baseline models 
models.append(('LR', LogisticRegression(max_iter=10000)))
models.append(('KNN', KNeighborsClassifier()))
models.append(('CART', DecisionTreeClassifier()))
models.append(('NB', GaussianNB()))
## ensemble models 
models.append(('RF', RandomForestClassifier()))
## boosting models 
models.append(('GBM', GradientBoostingClassifier()))
models.append(('AB', AdaBoostClassifier(algorithm="SAMME")))
## neural networks
models.append(('NN', MLPClassifier()))

In [4]:
results = []
names = []

kfold = StratifiedKFold(n_splits=5)
for name, model in models :
    cv_results = cross_val_score(model, 
                                 X_train, 
                                 y_train.values.ravel(),
                                 cv=kfold, 
                                 scoring='accuracy')
    results.append(cv_results)
    names.append(name)
    print(f"{name}: {round(cv_results.mean(), 4)} ({round(cv_results.std(), 6)})")

LR: 0.8734 (0.000921)
KNN: 0.83 (0.000809)
CART: 0.8754 (0.00106)
NB: 0.822 (0.001115)
RF: 0.8961 (0.000562)
GBM: 0.8958 (0.000408)
AB: 0.8786 (0.001366)
NN: 0.8954 (0.001303)


Save the results so they can be loaded later. 

In [5]:
cv_scores = {name:result for (name,result) in zip(names,results)}
dump(cv_scores, open("../models/cv_scores.p", "wb"))

In [6]:
cv_scores = load(open("../models/cv_scores.p", "rb" ))

We plot the results for each model below: 